In [ ]:
import cv2
import numpy as np

In [ ]:
def find_image_coordinates(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Create a window and display the image for manual selection
    cv2.namedWindow("Image")
    cv2.imshow("Image", image)
    cv2.waitKey(0)

    # Initialize an empty list to store the image coordinates
    image_points = []

    # Mouse callback function for manual selection
    def select_point(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            # Append the image coordinates of the selected point to the list
            image_points.append((x, y))
            # Draw a red dot at the selected point
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
            # Display the updated image
            cv2.imshow("Image", image)

    # Register the mouse callback function
    cv2.setMouseCallback("Image", select_point)

    # Wait for the user to close the image window
    cv2.waitKey(0)

    # Close all OpenCV windows
    cv2.destroyAllWindows()

    return image_points

In [ ]:
def estimate_fundamental_matrix(image_points1, image_points2):
    # Convert image points to homogeneous coordinates
    image_points1_homo = np.hstack((image_points1, np.ones((len(image_points1), 1))))
    image_points2_homo = np.hstack((image_points2, np.ones((len(image_points2), 1))))

    # Normalize image points
    image_points1_normalized = cv2.normalize(image_points1_homo, None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    image_points2_normalized = cv2.normalize(image_points2_homo, None, alpha=-1, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    # Estimate fundamental matrix using normalized eight-point algorithm
    F, _ = cv2.findFundamentalMat(image_points1_normalized, image_points2_normalized, cv2.FM_8POINT)

    return F

In [ ]:
def estimate_homography(image_points1, image_points2):
    # Construct the matrix A
    num_points = image_points1.shape[0]
    A = np.zeros((2*num_points, 9))
    for i in range(num_points):
        x1, y1 = image_points1[i]
        x2, y2 = image_points2[i]
        A[2*i] = [-x1, -y1, -1, 0, 0, 0, x2*x1, x2*y1, x2]
        A[2*i+1] = [0, 0, 0, -x1, -y1, -1, y2*x1, y2*y1, y2]

    # Perform singular value decomposition
    _, _, V = np.linalg.svd(A)

    # Extract the homography matrix H from the last column of V
    H = V[-1].reshape(3, 3)

    return H

In [ ]:
def triangulate_points(image_points1, image_points2, K1, K2, R1, R2, Ce1, Ce2):
    num_points = image_points1.shape[0]
    points_3d = np.zeros((num_points, 3))

    # Camera matrices for the camera pair
    P = K1 @ np.hstack((R1, -R1 @ Ce1.reshape(3, 1)))
    P_prime = K2 @ np.hstack((R2, -R2 @ Ce2.reshape(3, 1)))

    for i in range(num_points):
        x1, y1 = image_points1[i]
        x2, y2 = image_points2[i]

        # Homogeneous image coordinates
        x1_homo = np.array([[x1], [y1], [1]])
        x2_homo = np.array([[x2], [y2], [1]])

        # Linear triangulation
        A = np.zeros((4, 4))
        A[0] = x1_homo[0] * P[2] - P[0]
        A[1] = x1_homo[1] * P[2] - P[1]
        A[2] = x2_homo[0] * P_prime[2] - P_prime[0]
        A[3] = x2_homo[1] * P_prime[2] - P_prime[1]

        _, _, V = np.linalg.svd(A)
        X_homo = V[-1]
        X_homo /= X_homo[3]  # Normalize by the fourth coordinate

        # Convert homogeneous coordinates to 3D coordinates
        X = X_homo[:3]

        # Store the 3D point
        points_3d[i] = X

    return points_3d

In [ ]:
# Task 1: Get the image coordinates of the points from both images
image_path1 = "house1.png"  # Replace with the actual path to the first image
image_path2 = "house2.png"  # Replace with the actual path to the second image

In [ ]:
image_points1 = find_image_coordinates(image_path1)
image_points2 = find_image_coordinates(image_path2)

In [ ]:
# Task 2: Reconstruct the fundamental matrix and triangulate 3D points
F = estimate_fundamental_matrix(image_points1, image_points2)

In [ ]:
# Assuming the camera matrices and camera centers for the first image are known
K1 = np.array([[focal_length, 0, principal_point_x],
                [0, focal_length, principal_point_y],
               [0, 0, 1]])  # Replace with the actual values
R1 = np.eye(3)  # Replace with the actual rotation matrix
Ce1 = np.array([0, 0, 0])  # Replace with the actual camera center

    # Assuming the camera matrices and camera centers for the second image are known
K2 = np.array([[focal_length, 0, principal_point_x],
               [0, focal_length, principal_point_y],
               [0, 0, 1]])  # Replace with the actual values
R2 = np.eye(3)  # Replace with the actual rotation matrix
Ce2 = np.array([baseline, 0, 0])  # Replace with the actual camera center

# Triangulate the 3D points
points_3d = triangulate_points(image_points1, image_points2, K1, K2, R1, R2, Ce1, Ce2)

In [ ]:
# Task 3: Recover the 3D homography H
# Assuming the camera matrices and camera centers for the canonical camera pair are known
K_hat = np.eye(3)  # Replace with the actual calibration matrix
R_hat = np.eye(3)  # Replace with the actual rotation matrix
Ce_hat = np.zeros(3)  # Replace with the actual camera center

H = estimate_homography(points_3d, coords_world)

In [ ]:
# Task 4: Triangulate the points using the correct camera pair
points_3d_correct = triangulate_points(image_points1, image_points2, K1, K2, R1, R2, Ce1, Ce2)

# Print the 3D coordinates of the points
print("Reconstructed 3D Coordinates (with canonical camera pair):")
for i in range(points_3d.shape[0]):
    X = points_3d[i]
    print("Point {}: {}".format(i+1, X))

print("\nReconstructed 3D Coordinates (with correct camera pair):")
for i in range(points_3d_correct.shape[0]):
    X = points_3d_correct[i]
    print("Point {}: {}".format(i+1, X))